<a href="https://colab.research.google.com/github/amien1410/colab-notebooks/blob/main/Revised_Booking_com_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title
!pip install "httpx[http2,brotli]" parsel
import difflib
import httpx
import requests
import random
import string
import time
import urllib.parse
from bs4 import BeautifulSoup

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 29.2 MB/s eta 0:00:00


In [2]:
# @title
headers2 = {
    "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding":"gzip, deflate, br",
    "Accept-Language":"en-US,en;q=0.9,id;q=0.8,fa;q=0.7,ar;q=0.6,ms;q=0.5,ja;q=0.4",
    "Cache-Control":"max-age=0",
    "Cookie": 'px_init=0; bkng_sso_session=e30; OptanonConsent=implicitConsentCountry=nonGDPR&implicitConsentDate=1688341835317; cors_js=1; g_state={"i_p":1689063805365,"i_l":3}; bkng_expired_hint=eyJib29raW5nX2dsb2JhbCI6W3sibG9naW5faGludCI6MzEzOTY4OTU3OH1dfQ; bkng_sso_ses=e30; pcm_consent=consentedAt%3D2023-10-05T01%3A23%3A23.659Z%26countryCode%3DID%26expiresAt%3D2024-04-02T01%3A23%3A23.659Z%26implicit%3Dfalse%26regionCode%3DKT%26regulation%3Dnone%26legacyRegulation%3Dnone%26consentId%3Db29ee419-b359-4dec-a21f-1c8402c65d3b%26analytical%3Dtrue%26marketing%3Dtrue; bkng_sso_auth=CAIQsOnuTRpykSzmv8JMEafw8OBQmEwHlYkQD1BE/07FzWiRhRk7T+Cl6VTgFnDCiL56Xay0/XGqcNUHKKqKRFihOdZ7YC//cVRbU2pPHoYkTkSQH/6vQU6aieTRoTmgSulVjhsbphBZoqDGAoy73r5buX1/UdTCAI6Z; bkng_sso_auth_1696469003=CAIQsOnuTRpykSzmv8JMEafw8OBQmEwHlYkQD1BE/07FzWiRhRk7T+Cl6VTgFnDCiL56Xay0/XGqcNUHKKqKRFihOdZ7YC//cVRbU2pPHoYkTkSQH/6vQU6aieTRoTmgSulVjhsbphBZoqDGAoy73r5buX1/UdTCAI6Z; pcm_verified=1; _pxhd=lqCYPkaoZw2EcwRodSEQcVNjv2dUrdyjcnLkfwtrjaZ3VJxRxhMb95aKJwm2UR5dS%2Fzyx6MoscUH6ZKhBh%2F-yg%3D%3D%3ASmhouaMR9crUa2OA5WPDA9cM258mYWqsIRAnsSjNEykR3Ts%2FTD9nXnekDYhvmiSAlrrmKyEWBpVLo3RW%2FxRWQt82gElCC9nT9fvqYOlg0F8%3D; bkng=11UmFuZG9tSVYkc2RlIyh9Yaa29%2F3xUOLbVA9iGwA%2BUSz7Vg8K6c%2FTKO%2BIrpV1ZQ2n%2B5FCqFNiuhx1xPFQ4WLBeRrLd6nyCRo5cvbNZvp%2FWSKPgAsnzPLn15YE4j0gtjbNgUmaPaznmATSU5ZXekDjP0VmKpof4OazBIAPTOmbG0%2FXze%2FPP45cuj4dAg0W1pjmSj5r4FlUS40G2Yy9xiPorA%3D%3D; lastSeen=0',
    "Sec-Ch-Ua":'"Not.A/Brand";v="8", "Chromium";v="114", "Microsoft Edge";v="114"',
    "Sec-Ch-Ua-Mobile":"?0",
    "Sec-Ch-Ua-Platform":"Windows",
    "Sec-Fetch-Dest":"document",
    "Sec-Fetch-Mode":"navigate",
    "Sec-Fetch-Site":"none",
    "Sec-Fetch-User":"?1",
    "Upgrade-Insecure-Requests":'1',
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}
# start HTTP session client with our headers and HTTP2
client = httpx.AsyncClient(
    http2=True,  # http2 connections are significantly less likely to get blocked
    headers=headers2,
    timeout=httpx.Timeout(15.0),
    limits=httpx.Limits(max_connections=5),
)

# get additional info from the property
def get_additional_info(url):
  results = []
  print(url)
  res = requests.get(url, headers=headers)
  soup = BeautifulSoup(res.text)
  address = soup.select("span.hp_address_subtitle")[0].text.strip()
  categories = soup.select(".c-score-bar")
  for i in range(len(categories)):
    title = categories[i].select("span")[0].text.strip()
    score = categories[i].select("span")[1].text
    results.append(f"{title} ({score})")
  results = ", ".join(results)
  print(address, results)
  return address, results

# get all reviews from the given property name
async def get_reviews(pageName):
  results = []
  hasNextPage = True
  offset = 0
  page = 1
  while hasNextPage == True:
    url = "https://www.booking.com/reviewlist.html?cc1=us&pagename="+pageName+"&rows=25&offset="+str(offset)
    print(url)
    res = await client.get(url)
    soup = BeautifulSoup(res.text)
    reviews = soup.select(".review_list_new_item_block")

    if len(reviews) < 25:
      hasNextPage = False

    for i in range(len(reviews)):
      countLike = reviews[i].select(".-iconset-review_great")
      countDislike = reviews[i].select(".-iconset-review_poor")
      countResponse = reviews[i].select(".-iconset-chat_bubbles")
      like = ""
      dislike = ""
      response = ""
      if len(countLike) > 0:
        like = countLike[0].parent.find_next_siblings()[1].text
      if len(countDislike) > 0:
        dislike = countDislike[0].parent.find_next_siblings()[1].text
      if len(countResponse) > 0:
        response = countResponse[0].parent.find_next_siblings()[0].text
      results.append({
          "username" : reviews[i].select('.bui-avatar-block__title')[0].text,
          "likeReview" : like,
          "dislikeReview" : dislike,
          "response" : response,
          "rating" : reviews[i].select(".bui-review-score__badge")[0].text.strip(),
          "reviewDate" : reviews[i].select(".c-review-block__date")[1].text.split(": ")[1].strip()
      })

    offset += 25
    page += 1
    time.sleep(2)

  return results

In [11]:
random_request_id = "".join(
    random.choice(string.ascii_lowercase + string.digits) for i in range(180)
)
headers = {
    "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding":"gzip, deflate, br",
    "Accept-Language":"en-US,en;q=0.9,id;q=0.8,fa;q=0.7,ar;q=0.6,ms;q=0.5,ja;q=0.4",
    "Cache-Control":"max-age=0",
    "Cookie": 'px_init=0; bkng_sso_session=e30; OptanonConsent=implicitConsentCountry=nonGDPR&implicitConsentDate=1688341835317; cors_js=1; g_state={"i_p":1689063805365,"i_l":3}; bkng_expired_hint=eyJib29raW5nX2dsb2JhbCI6W3sibG9naW5faGludCI6MzEzOTY4OTU3OH1dfQ; bkng_sso_ses=e30; pcm_consent=consentedAt%3D2023-10-05T01%3A23%3A23.659Z%26countryCode%3DID%26expiresAt%3D2024-04-02T01%3A23%3A23.659Z%26implicit%3Dfalse%26regionCode%3DKT%26regulation%3Dnone%26legacyRegulation%3Dnone%26consentId%3Db29ee419-b359-4dec-a21f-1c8402c65d3b%26analytical%3Dtrue%26marketing%3Dtrue; bkng_sso_auth=CAIQsOnuTRpykSzmv8JMEafw8OBQmEwHlYkQD1BE/07FzWiRhRk7T+Cl6VTgFnDCiL56Xay0/XGqcNUHKKqKRFihOdZ7YC//cVRbU2pPHoYkTkSQH/6vQU6aieTRoTmgSulVjhsbphBZoqDGAoy73r5buX1/UdTCAI6Z; bkng_sso_auth_1696469003=CAIQsOnuTRpykSzmv8JMEafw8OBQmEwHlYkQD1BE/07FzWiRhRk7T+Cl6VTgFnDCiL56Xay0/XGqcNUHKKqKRFihOdZ7YC//cVRbU2pPHoYkTkSQH/6vQU6aieTRoTmgSulVjhsbphBZoqDGAoy73r5buX1/UdTCAI6Z; pcm_verified=1; _pxhd=lqCYPkaoZw2EcwRodSEQcVNjv2dUrdyjcnLkfwtrjaZ3VJxRxhMb95aKJwm2UR5dS%2Fzyx6MoscUH6ZKhBh%2F-yg%3D%3D%3ASmhouaMR9crUa2OA5WPDA9cM258mYWqsIRAnsSjNEykR3Ts%2FTD9nXnekDYhvmiSAlrrmKyEWBpVLo3RW%2FxRWQt82gElCC9nT9fvqYOlg0F8%3D; bkng=11UmFuZG9tSVYkc2RlIyh9Yaa29%2F3xUOLbVA9iGwA%2BUSz7Vg8K6c%2FTKO%2BIrpV1ZQ2n%2B5FCqFNiuhx1xPFQ4WLBeRrLd6nyCRo5cvbNZvp%2FWSKPgAsnzPLn15YE4j0gtjbNgUmaPaznmATSU5ZXekDjP0VmKpof4OazBIAPTOmbG0%2FXze%2FPP45cuj4dAg0W1pjmSj5r4FlUS40G2Yy9xiPorA%3D%3D; lastSeen=0',
    "Origin": "https://www.booking.com",
    "Referer": "https://www.booking.com/searchresults.html?",
    "Sec-Ch-Ua":'"Microsoft Edge";v="117", "Not;A=Brand";v="8", "Chromium";v="117"',
    "Sec-Ch-Ua-Mobile":"?0",
    "Sec-Ch-Ua-Platform":"Windows",
    "Sec-Fetch-Dest":"document",
    "Sec-Fetch-Mode":"navigate",
    "Sec-Fetch-Site":"none",
    "Sec-Fetch-User":"?1",
    "Upgrade-Insecure-Requests":'1',
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
    "X-Requested-By": random_request_id,
}
# start HTTP session client with our headers and HTTP2
client = httpx.AsyncClient(
    http2=True,  # http2 connections are significantly less likely to get blocked
    headers=headers,
    timeout=httpx.Timeout(15.0),
    limits=httpx.Limits(max_connections=5),
)

In [3]:
# get 20 top properties from the search page
headers = {
    "Host": "www.booking.com",
    "Method": "GET",
    "Path": "/searchresults.html?ss=Aston+Banua+Banjarmasin",
    "Scheme": "https",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9,id;q=0.8,fa;q=0.7,ar;q=0.6,ms;q=0.5,ja;q=0.4",
    "Cache-Control": "max-age=0",
    "Cookie": "px_init=0; bkng_sso_session=e30; OptanonConsent=implicitConsentCountry=nonGDPR&implicitConsentDate=1688341835317; cors_js=1; g_state={\"i_p\":1689063805365,\"i_l\":3}; bkng_expired_hint=eyJib29raW5nX2dsb2JhbCI6W3sibg9naW5faGludCI6MzEzOTY4OTU3OH1dfQ; bkng_sso_ses=e30; pcm_consent=consentedAt%3D2023-10-05T01%3A23%3A23.659Z%26countryCode%3DID%26expiresAt%3D2024-04-02T01%3A23%3A23.659Z%26implicit%3Dfalse%26regionCode%3DKT%26regulation%3Dnone%26legacyRegulation%3Dnone%26consentId%3Db29ee419-b359-4dec-a21f-1c8402c65d3b%26analytical%3Dtrue%26marketing%3Dtrue; bkng_sso_auth=CAIQsOnuTRpykSzmv8JMEafw8OBQmEwHlYkQD1BE/07FzWiRhRk7T+Cl6VTgFnDCiL56Xay0/XGqcNUHKKqKRFihOdZ7YC//cVRbU2pPHoYkTkSQH/6vQU6aieTRoTmgSulVjhsbphBZoqDGAoy73r5buX1/UdTCAI6Z; bkng_sso_auth_1696469003=CAIQsOnuTRpykSzmv8JMEafw8OBQmEwHlYkQD1BE/07FzWiRhRk7T+Cl6VTgFnDCiL56Xay0/XGqcNUHKKqKRFihOdZ7YC//cVRbU2pPHoYkTkSQH/6vQU6aieTRoTmgSulVjhsbphBZoqDGAoy73r5buX1/UdTCAI6Z; pcm_verified=1; _pxhd=lqCYPkaoZw2EcwRodSEQcVNjv2dUrdyjcnLkfwtrjaZ3VJxRxhMb95aKJwm2UR5dS%2Fzyx6MoscUH6ZKhBh%2F-yg%3D%3D%3ASmhouaMR9crUa2OA5WPDA9cM258mYWqsIRAnsSjNEykR3Ts%2FTD9nXnekDYhvmiSAlrrmKyEWBpVLo3RW%2FxRWQt82gElCC9nT9fvqYOlg0F8%3D; bkng=11UmFuZG9tSVYkc2RlIyh9Yaa29%2F3xUOLbVA9iGwA%2BUSz7Vg8K6c%2FTKO%2BIrpV1ZQ2n%2B5FCqFNiuhx1xPFQ4WLBeRrLd6nyCRo5cvbNZvp%2FWSKPgAsnzPLn15YE4j0gtjbNgUmaPaznmATSU5ZXekDjP0VmKpof4OazBIAPTOmbG0%2FXze%2FPP45cuj4dAg0W1pjmSj5r4FlUS40G2Yy9xiPorA%3D%3D; lastSeen=0",
    "Sec-Ch-Ua": "\"Microsoft Edge\";v=\"117\", \"Not;A=Brand\";v=\"8\", \"Chromium\";v=\"117\"",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "\"Windows\"",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

headers2 = {
    "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding":"gzip, deflate, br",
    "Accept-Language":"en-US,en;q=0.9,id;q=0.8,fa;q=0.7,ar;q=0.6,ms;q=0.5,ja;q=0.4",
    "Cache-Control":"max-age=0",
    "Cookie": 'px_init=0; bkng_sso_session=e30; OptanonConsent=implicitConsentCountry=nonGDPR&implicitConsentDate=1688341835317; cors_js=1; g_state={"i_p":1689063805365,"i_l":3}; bkng_expired_hint=eyJib29raW5nX2dsb2JhbCI6W3sibG9naW5faGludCI6MzEzOTY4OTU3OH1dfQ; bkng_sso_ses=e30; pcm_consent=consentedAt%3D2023-10-05T01%3A23%3A23.659Z%26countryCode%3DID%26expiresAt%3D2024-04-02T01%3A23%3A23.659Z%26implicit%3Dfalse%26regionCode%3DKT%26regulation%3Dnone%26legacyRegulation%3Dnone%26consentId%3Db29ee419-b359-4dec-a21f-1c8402c65d3b%26analytical%3Dtrue%26marketing%3Dtrue; bkng_sso_auth=CAIQsOnuTRpykSzmv8JMEafw8OBQmEwHlYkQD1BE/07FzWiRhRk7T+Cl6VTgFnDCiL56Xay0/XGqcNUHKKqKRFihOdZ7YC//cVRbU2pPHoYkTkSQH/6vQU6aieTRoTmgSulVjhsbphBZoqDGAoy73r5buX1/UdTCAI6Z; bkng_sso_auth_1696469003=CAIQsOnuTRpykSzmv8JMEafw8OBQmEwHlYkQD1BE/07FzWiRhRk7T+Cl6VTgFnDCiL56Xay0/XGqcNUHKKqKRFihOdZ7YC//cVRbU2pPHoYkTkSQH/6vQU6aieTRoTmgSulVjhsbphBZoqDGAoy73r5buX1/UdTCAI6Z; pcm_verified=1; _pxhd=lqCYPkaoZw2EcwRodSEQcVNjv2dUrdyjcnLkfwtrjaZ3VJxRxhMb95aKJwm2UR5dS%2Fzyx6MoscUH6ZKhBh%2F-yg%3D%3D%3ASmhouaMR9crUa2OA5WPDA9cM258mYWqsIRAnsSjNEykR3Ts%2FTD9nXnekDYhvmiSAlrrmKyEWBpVLo3RW%2FxRWQt82gElCC9nT9fvqYOlg0F8%3D; bkng=11UmFuZG9tSVYkc2RlIyh9Yaa29%2F3xUOLbVA9iGwA%2BUSz7Vg8K6c%2FTKO%2BIrpV1ZQ2n%2B5FCqFNiuhx1xPFQ4WLBeRrLd6nyCRo5cvbNZvp%2FWSKPgAsnzPLn15YE4j0gtjbNgUmaPaznmATSU5ZXekDjP0VmKpof4OazBIAPTOmbG0%2FXze%2FPP45cuj4dAg0W1pjmSj5r4FlUS40G2Yy9xiPorA%3D%3D; lastSeen=0',
    "Sec-Ch-Ua":'"Not.A/Brand";v="8", "Chromium";v="114", "Microsoft Edge";v="114"',
    "Sec-Ch-Ua-Mobile":"?0",
    "Sec-Ch-Ua-Platform":"Windows",
    "Sec-Fetch-Dest":"document",
    "Sec-Fetch-Mode":"navigate",
    "Sec-Fetch-Site":"none",
    "Sec-Fetch-User":"?1",
    "Upgrade-Insecure-Requests":'1',
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}
# start HTTP session client with our headers and HTTP2
client = httpx.AsyncClient(
    http2=True,  # http2 connections are significantly less likely to get blocked
    headers=headers,
    timeout=httpx.Timeout(15.0),
    limits=httpx.Limits(max_connections=5),
)

def get_hotels(hotel_name, address):
  results = []
  query = urllib.parse.quote(hotel_name+" "+address)
  url = "https://www.booking.com/searchresults.html?ss=Aston+Banua+Banjarmasin"
  # url = "https://www.booking.com/searchresults.html?ss=Aston+Banua+Banjarmasin"
  res = requests.get(url, headers=headers2)
  soup = BeautifulSoup(res.text, 'lxml')
  # properties = soup.select("a.e13098a59f")
  return soup
  # for i in range(len(properties)):
  #   url = properties[i].get("href")
  #   pageName = url.split("/")[-1].split(".")[0]
  #   results.append({
  #       "newPropertyName": properties[i].find("div").text,
  #       "pageName": pageName,
  #       "url": url
  #   })

  #   if i > 19:
  #     return results

In [12]:
hotels = get_hotels("Residence Inn", "410 North Chicago")
hotels.select("h3")

[<h3 class="b87c397a13 fb2f406ff9" data-testid="promotional-banner-content-title">Quick escape, quality time</h3>,
 <h3 class="b87c397a13 fb2f406ff9" data-testid="promotional-banner-content-title">Skip the lines, not the memories</h3>,
 <h3 class="b87c397a13 fb2f406ff9" data-testid="promotional-banner-content-title">Genius flight price alerts</h3>,
 <h3 class="b87c397a13 fb2f406ff9" data-testid="promotional-banner-content-title">Five seater with booster seats?</h3>,
 <h3 class="b87c397a13 fb2f406ff9" data-testid="promotional-banner-content-title">Parenting is better with a view</h3>,
 <h3 class="travel_articles__header travel_articles__two_line" id="travel_articles_title_1">5 of the best hotels in Los Angeles</h3>,
 <h3 class="e7addce19e b7dfb6cc1c" id=":R6d2:">Book now, pay at the property</h3>,
 <h3 class="e7addce19e b7dfb6cc1c" id=":Rad2:">2+ million properties worldwide</h3>,
 <h3 class="e7addce19e b7dfb6cc1c" id=":Red2:">Trusted 24/7 customer service you can rely on</h3>,
 <h3 cla

In [24]:
def getHotels(hotelName, address, city):
  results = []
  query = urllib.parse.quote(hotelName+" "+address)
  destType = getDestId(city)
  url = "https://www.booking.com/searchresults.html?ss="+query+"&dest_id="+destType+"&dest_type=hotel"
  print(url)
  res = requests.get(url, headers=headers)
  soup = BeautifulSoup(res.text, 'lxml')
  properties = soup.select("a.bd77474a8e")
  print(properties)
  for i in range(len(properties)):
    url = properties[i].get("href")
    pageName = url.split("/")[-1].split(".")[0]
    results.append({
        "newPropertyName": properties[i].find("div").text,
        "pageName": pageName,
        "url": url
    })

  return results

In [14]:
headers3 = {
    "Accept":"*/*",
    "Accept-Encoding":"gzip, deflate, br",
    "Accept-Language":"en-US,en;q=0.9,id;q=0.8,fa;q=0.7,ar;q=0.6,ms;q=0.5,ja;q=0.4",
    "Content-Length":"97",
    "Content-Type":"text/plain;charset=UTF-8",
    "Cookie":"px_init=0; bkng_sso_session=e30; cors_js=1; bkng_expired_hint=eyJib29raW5nX2dsb2JhbCI6W3sibG9naW5faGludCI6MzEzOTY4OTU3OH1dfQ; bkng_sso_ses=e30; pcm_consent=consentedAt%3D2023-10-05T01%3A23%3A23.659Z%26countryCode%3DID%26expiresAt%3D2024-04-02T01%3A23%3A23.659Z%26implicit%3Dfalse%26regionCode%3DKT%26regulation%3Dnone%26legacyRegulation%3Dnone%26consentId%3Db29ee419-b359-4dec-a21f-1c8402c65d3b%26analytical%3Dtrue%26marketing%3Dtrue; bkng_sso_auth=CAIQsOnuTRpykSzmv8JMEafw8OBQmEwHlYkQD1BE/07FzWiRhRk7T+Cl6VTgFnDCiL56Xay0/XGqcNUHKKqKRFihOdZ7YC//cVRbU2pPHoYkTkSQH/6vQU6aieTRoTmgSulVjhsbphBZoqDGAoy73r5buX1/UdTCAI6Z; g_state={'i_p':1698891354300,'i_l':4}; _vs=4496662336926575762:1696476352.8664062:8139239005403879237; 11_srd=%7B%22features%22%3A%5B%7B%22id%22%3A16%7D%5D%2C%22score%22%3A3%2C%22detected%22%3Afalse%7D; _pxhd=5ScEwaqD-7M3tg9IQbg4inyqSLYDbs4aP6KKxtqCUEQyrd8dY8htNPOpNlyFb96Aw56CNB52WbSGAsSqcsEwnw%3D%3D%3A4JFiFATPLE5uHJRl4WH%2FfwQAhZkOfzmsErRiH1zZtamB0rQHUox3ySvAkXgCOIcjOHA%2F5JdLidI5mVgo%2F71mjPmzQ3n8WJCNaswHzTEQf98%3D; bkng=11UmFuZG9tSVYkc2RlIyh9Yaa29%2F3xUOLblgO%2Fz4BDP5vo0AnnJEYr%2BDF1TOIpdko7ZUwuj05DOowpzvFt7fumKjnXxR4fIIIrygGcufWequ77aOTsT9NzXV%2FrOzvcN5h9zX7yyZI1qVrHKg2VYOhdCMQ04NU562w%2BbLAUy66oXtJOXzG%2BKPHVZlxPIb1CMIYpjUu923TUTKrLMTDsthSTFA%3D%3D; lastSeen=0",
    "Origin":"https://www.booking.com",
    "Referer":"https://www.booking.com/",
    "Sec-Ch-Ua":'"Microsoft Edge";v="117", "Not;A=Brand";v="8", "Chromium";v="117"',
    "Sec-Ch-Ua-Mobile":"?0",
    "Sec-Ch-Ua-Platform":"Windows",
    "Sec-Fetch-Dest":"empty",
    "Sec-Fetch-Mode":"cors",
    "Sec-Fetch-Site":"same-site",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
def getDestId(city):
  try:
    url = "https://accommodations.booking.com/autocomplete.json"
    params = {"query":city,"pageview_id":"bd241b165c8a020d","aid":304142,"language":"en-us","size":5}
    res = requests.post(url, json=params)
    data = res.json()
    res = []
    if len(data['results']) == 0:
      return {"No result, check your query!"}
    for result in data.get("results", []):
      if result.get("dest_type") == "city":
        res.append(result['dest_id'])
    if len(res) > 0:
      return res[0]
    else:
      return {"There is no destination city type from the result"}
  except:
    return {"Error!"}

In [25]:
# getDestId("Jakarta")
getHotels("Residence Inn", "410 North", "Chicago")
# getHotels("TreePark", "", "Banjarmasin")

https://www.booking.com/searchresults.html?ss=Residence%20Inn%20410%20North&dest_id=20033173&dest_type=hotel
[<a class="bd77474a8e" data-testid="title-link" href="https://www.booking.com/hotel/us/the-allerton.html?aid=304142&amp;label=gen173bo-1DCAQoggJCDmhvdGVsXzIwMDMzMTczSDNYA2imAogBAZgBMbgBGMgBDNgBA-gBAfgBA4gCAZgCAqgCBLgC4_DMwQbAAgHSAiQ4ZjFkYzU2YS1hMWVlLTRjMTAtYWQ0OC1kMGZjZTBjOGNhYmTYAgTgAgE&amp;ucfs=1&amp;arphpl=1&amp;dest_id=20033173&amp;dest_type=hotel&amp;group_adults=2&amp;req_adults=2&amp;no_rooms=1&amp;group_children=0&amp;req_children=0&amp;hpos=1&amp;hapos=1&amp;sr_order=popularity&amp;srpvid=a3d46d71809f024b&amp;srepoch=1748187236&amp;from=searchresults" rel="noopener noreferrer" target="_blank"><div class="b87c397a13 a3e0b4ffd1" data-testid="title">Warwick Allerton Chicago</div><div class="bc946a29db">Opens in new window</div></a>, <a class="bd77474a8e" data-testid="title-link" href="https://www.booking.com/hotel/us/riu-plaza-chicago.html?aid=304142&amp;label=gen173bo-1DC

[{'newPropertyName': 'Warwick Allerton Chicago',
  'pageName': 'the-allerton',
  'url': 'https://www.booking.com/hotel/us/the-allerton.html?aid=304142&label=gen173bo-1DCAQoggJCDmhvdGVsXzIwMDMzMTczSDNYA2imAogBAZgBMbgBGMgBDNgBA-gBAfgBA4gCAZgCAqgCBLgC4_DMwQbAAgHSAiQ4ZjFkYzU2YS1hMWVlLTRjMTAtYWQ0OC1kMGZjZTBjOGNhYmTYAgTgAgE&ucfs=1&arphpl=1&dest_id=20033173&dest_type=hotel&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=a3d46d71809f024b&srepoch=1748187236&from=searchresults'},
 {'newPropertyName': 'Riu Plaza Chicago',
  'pageName': 'riu-plaza-chicago',
  'url': 'https://www.booking.com/hotel/us/riu-plaza-chicago.html?aid=304142&label=gen173bo-1DCAQoggJCDmhvdGVsXzIwMDMzMTczSDNYA2imAogBAZgBMbgBGMgBDNgBA-gBAfgBA4gCAZgCAqgCBLgC4_DMwQbAAgHSAiQ4ZjFkYzU2YS1hMWVlLTRjMTAtYWQ0OC1kMGZjZTBjOGNhYmTYAgTgAgE&ucfs=1&arphpl=1&dest_id=20033173&dest_type=hotel&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=2&hapos=2&sr

In [9]:
new_address, categories = get_additional_info(hotels[0]['url'])

KeyError: 0

In [ ]:
await get_reviews("residence-inn-chicago-downtown-river-north")